In [124]:
import pandas as pd
import pickle
from typing import List
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from category_encoders.target_encoder import TargetEncoder
import warnings
warnings.filterwarnings("ignore")

class DataPrep:

    df: pd.DataFrame
    metadata: str
    col_names: List[str]

    def __init__(self, data_path: str):
        self.data_path = data_path
        self.inhale_metadata()

    def create_dataframe(self, print_=False):
        self.df = pd.read_csv(self.data_path, header=None)
        if print_:
            print(self.df.head(3))

    def inhale_metadata(self):
        with open('./census_data/census_income_metadata.txt', 'r') as f:
            self.metadata = f.read()

    def get_col_names(self, print_=False):
        self.col_names = [attr.split(':')[0] for attr in self.metadata.split('- 50000, 50000+.')[1].strip().split('\n')]
        self.col_names.remove('| instance weight'), self.col_names.append('target')
        if print_:
            print(self.col_names)

    def set_col_names(self, print_=False):
        self.df.columns = self.col_names
        if print_:
            print(self.df.head())

    def feature_engineering(self):
        """metadata indicated this feature should be dropped"""
        self.df.drop('instance weight', axis=1, inplace=True)

        """a single individual can't have both a capital gain AND a capital loss so condense those"""
        self.df['capital_change'] = self.df['capital gains'] - self.df['capital losses']
        self.df.drop(['capital gains', 'capital losses'], axis=1, inplace=True)

        """replace the target feature string values with integers"""
        self.df['target'].replace({' - 50000.': -1, ' 50000+.': 1}, inplace=True)
        
    def numerical_scaling(self, data: pd.DataFrame):
        scaler = StandardScaler()
        return scaler.fit_transform(data)
    
    def categorical_encoding(self, data: pd.DataFrame, target: pd.Series = None, scheme: str = 'target'):
        if scheme in ['one_hot', 'label']:
            encoder = OneHotEncoder() if scheme == 'one_hot' else LabelEncoder()
            return encoder.fit_transform(data)
        else:
            encoder = TargetEncoder()
            encoder.fit(X=data, y=target)
            return encoder.transform(data)
    
    def get_val_counts(self, cols: List[str]=None):
        if cols is None: cols = self.df.columns
        for col in cols:
                print(f'{col}: \n{self.df[col].value_counts()}\n')

    def apply_all(self, filename: str): #####TODO: add all methods to this method
        self.create_dataframe()
        if self.metadata is None: self.inhale_metadata()
        self.get_col_names()
        self.set_col_names()

        self.pickle_data(filename)


    def pickle_data(self, filename: str):
        with open(f'./pickles/{filename}.obj', 'wb') as f:
            pickle.dump(self.df, f)

In [125]:
train_data = DataPrep('./census_data/census_income_learn.csv')

In [126]:
train_data.create_dataframe(print_=True)

   0                                1   2   3                            4   \
0  73                  Not in universe   0   0         High school graduate   
1  58   Self-employed-not incorporated   4  34   Some college but no degree   
2  18                  Not in universe   0   0                   10th grade   

   5                 6               7                             8   \
0   0   Not in universe         Widowed   Not in universe or children   
1   0   Not in universe        Divorced                  Construction   
2   0       High school   Never married   Not in universe or children   

                                     9   ...              32              33  \
0                       Not in universe  ...   United-States   United-States   
1   Precision production craft & repair  ...   United-States   United-States   
2                       Not in universe  ...         Vietnam         Vietnam   

               34                                    35 36           

In [127]:
train_data.inhale_metadata()
train_data.get_col_names(print_=True)

['age', 'class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'wage per hour', 'enroll in edu inst last wk', 'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 'full or part time employment stat', 'capital gains', 'capital losses', 'dividends from stocks', 'tax filer stat', 'region of previous residence', 'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', 'instance weight', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits', 'week

In [128]:
train_data.set_col_names(print_=True)

   age                  class of worker  detailed industry recode  \
0   73                  Not in universe                         0   
1   58   Self-employed-not incorporated                         4   
2   18                  Not in universe                         0   
3    9                  Not in universe                         0   
4   10                  Not in universe                         0   

   detailed occupation recode                    education  wage per hour  \
0                           0         High school graduate              0   
1                          34   Some college but no degree              0   
2                           0                   10th grade              0   
3                           0                     Children              0   
4                           0                     Children              0   

  enroll in edu inst last wk    marital stat           major industry code  \
0            Not in universe         Widowed

In [129]:
train_data.feature_engineering()

In [130]:
train_data.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 40 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   age                                         199523 non-null  int64 
 1   class of worker                             199523 non-null  object
 2   detailed industry recode                    199523 non-null  int64 
 3   detailed occupation recode                  199523 non-null  int64 
 4   education                                   199523 non-null  object
 5   wage per hour                               199523 non-null  int64 
 6   enroll in edu inst last wk                  199523 non-null  object
 7   marital stat                                199523 non-null  object
 8   major industry code                         199523 non-null  object
 9   major occupation code                       199523 non-null  object
 10  race    

In [131]:
data_to_scale = train_data.df[['age', 'wage per hour', 'dividends from stocks', 'weeks worked in year', 'capital_change']]
data_to_scale = train_data.numerical_scaling(data_to_scale)
train_data.df[['age', 'wage per hour', 'dividends from stocks', 'weeks worked in year', 'capital_change']] = data_to_scale
train_data.df.head(3)

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,...,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,target,capital_change
0,1.725879,Not in universe,0,0,High school graduate,-0.201629,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,Native- Born in the United States,0,Not in universe,2,-0.949346,95,-1,-0.084396
1,1.053560,Self-employed-not incorporated,4,34,Some college but no degree,-0.201629,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,Native- Born in the United States,0,Not in universe,2,1.180804,94,-1,-0.084396
2,-0.739291,Not in universe,0,0,10th grade,-0.201629,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,-0.949346,95,-1,-0.084396


In [132]:
data_to_encode = train_data.df[['class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'enroll in edu inst last wk', 'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 'full or part time employment stat', 'tax filer stat', 'region of previous residence', 'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits', 'year']]
target = train_data.df['target']
data_to_encode = train_data.categorical_encoding(train_data.df[['class of worker', 'detailed industry recode']], scheme='target')
train_data.df[['class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'enroll in edu inst last wk', 'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 'full or part time employment stat', 'tax filer stat', 'region of previous residence', 'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits', 'year']] = data_to_encode
train_data.df.head(3)

ValueError: Supervised encoders need a target for the fitting. The target cannot be None

In [ ]:
train_data.get_val_counts()

In [ ]:
train_data.df.columns

In [ ]:
# train_data.pickle_data('df_train')

In [ ]:
# test_data = DataPrep('./census_data/census_income_test.csv')
# test_data.apply_all('df_test')